In [1]:
import pandas as pd

TEST = '/kaggle/input/playground-series-s4e4/test.csv'
TRAIN = '/kaggle/input/playground-series-s4e4/train.csv'
test_df = pd.read_csv(filepath_or_buffer=TEST, index_col=['id']).drop(columns=['Sex'])

train_df = pd.read_csv(filepath_or_buffer=TRAIN, index_col=['id']).drop(columns=['Sex'])
train_df.head()

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
id,,,,,,,,
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [2]:
import warnings
from plotly import express

warnings.filterwarnings(action='ignore', category=FutureWarning)
express.histogram(data_frame=train_df, x='Rings', )

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

COLUMNS = ['Length', 'Diameter', 'Height', 'Whole weight', 'Whole weight.1', 'Whole weight.2', 'Shell weight',]
TARGET = 'Rings'

X_train, X_test, y_train, y_test = train_test_split(train_df[COLUMNS], train_df[TARGET], test_size=0.2, random_state=2024)

Let's make a simple Keras regression model and be done.

In [4]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input


def baseline_model():
    model = Sequential()
    model.add(Input((7, )))
    model.add(BatchNormalization())
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['mae', 'mse']) 
    return model

2024-04-16 17:32:03.733253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 17:32:03.733399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 17:32:03.936307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
model = baseline_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 7)              │            28 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,245 (4.86 KB)

 Trainable params: 1,231 (4.81 KB)

 Non-trainable params: 14 (56.00 B)

In [6]:
history = model.fit(X_train, y_train, epochs=100, verbose=1, validation_split = 0.2)

Epoch 1/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 28.6621 - mae: 3.8245 - mse: 28.6621 - val_loss: 3.9995 - val_mae: 1.3378 - val_mse: 4.0051
Epoch 2/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 6.5856 - mae: 1.8622 - mse: 6.5856 - val_loss: 4.3968 - val_mae: 1.3358 - val_mse: 4.4041
Epoch 3/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 6.1010 - mae: 1.7834 - mse: 6.1010 - val_loss: 3.8096 - val_mae: 1.2994 - val_mse: 3.8141
Epoch 4/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 6.0429 - mae: 1.7545 - mse: 6.0429 - val_loss: 3.8146 - val_mae: 1.3403 - val_mse: 3.8191
Epoch 5/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 5.5939 - mae: 1.6802 - mse: 5.5939 - val_loss: 3.9118 - val_mae: 1.3163 - val_mse: 3.9169
Epoch 6/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 5.4120 - mae: 1.6473 - mse: 5.4120 - val_loss: 3.9026 - val_mae: 1.3695 - val_mse: 3.9058
Epoch 7/100
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 5.3460 - mae: 1.6

In [7]:
history_df = pd.DataFrame(data=history.history)
history_df['epoch'] = history.epoch
history_df.tail()

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
95,4.637828,1.498677,4.636996,4.197662,1.368168,4.202392,95
96,4.596837,1.494190,4.597156,3.945825,1.364196,3.949301,96
97,4.657223,1.498151,4.658932,3.937424,1.383091,3.940202,97
98,4.636128,1.496844,4.630192,4.033042,1.370749,4.037171,98
99,4.632671,1.495595,4.632966,3.952706,1.372029,3.956157,99


In [8]:
from plotly import express
express.line(data_frame=history_df, x='epoch', y=['mae', 'val_mae']).show()
express.line(data_frame=history_df, x='epoch', y=['loss', 'val_loss']).show()

In [9]:
import numpy as np
best_df = pd.DataFrame(data={'id' : test_df.index.tolist(), 'Rings': model.predict(test_df[COLUMNS]).flatten()})
best_df.to_csv(path_or_buf='/kaggle/working/submission.csv', index=False)

1888/1888 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
